In [ ]:
%pylab inline
import os
import shutil
from glob import glob
import pysumma as ps
import xarray as xr
import pandas as pd
from pathlib import Path

In [ ]:
veg_igbp = pd.read_csv('./VEGPARM_IGBP_MODIS_NOAH.TBL', 
                       index_col=-1, skipinitialspace=True)
veg_igbp.index = veg_igbp.index.map(lambda x: x.strip().replace("'", ""))

soil_rosetta = pd.read_csv('./SOILPARM_ROSETTA.TBL', 
                           index_col=-1, skipinitialspace=True)
soil_rosetta.index = soil_rosetta.index.map(lambda x: x.strip().replace("'", ""))

In [ ]:
sim_sites = os.listdir('../sites/')

model_src = f'../new_models/all_var_dense_dropout.txt'
model_rel_dest = f'../params/all_var_dense_dropout.txt'
executable = '../lrp_ml_summa/bin/lrp_summa'

config = {site: {'file_manager': f'../sites/{site}/file_manager.txt'} for site in sim_sites}

for s, c in config.items():
    # set model in file manager
    fman = ps.FileManager(c['file_manager'], name='')
    fman.options.append(ps.file_manager.FileManagerOption('neuralNetFile', model_rel_dest))
    fman['outFilePrefix'] = f'lrp_nn_output'
    
    # Create a new initial conditions
    fman['initConditionFile'] = f'../params/state_initial_conditions.nc'
    
    # save file manager
    fman.write(path=Path(str(fman.original_path).replace('file_manager', f'file_manager_lrp')))
    
    # copy neural net file over to params
    nn_dest = c['file_manager'].replace('file_manager.txt', model_rel_dest.replace('../', ''))
    shutil.copy(model_src, nn_dest)
    c['file_manager'] = f'../sites/{s}/file_manager_lrp.txt'

In [ ]:
ens = ps.Ensemble(executable, config, num_workers=31)
ens.run('local')
summary = ens.summary()

In [ ]:
summary = ens.summary()

In [ ]:
failed = summary['error']
success = summary['success']

len(failed), len(success)

In [ ]:
ens.rerun_failed('local')

In [ ]:
ens.summary()['error']